In [1]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import backend as BK


In [2]:
if  BK.image_data_format() == 'channels_first':
    input_shape = (3,224,224)
else: 
    input_shape = (224,224,3)

In [3]:
train_path = 'Emotion/Train'
valid_path = 'Emotion/Validation'

In [4]:
vgg = VGG16(weights = 'imagenet',include_top=False, input_shape=input_shape)

In [5]:
for layer in vgg.layers:
    layer.trainable = False

In [6]:
from glob import glob
folders = glob(train_path+'/*')
folders

['Emotion/Train\\Angry',
 'Emotion/Train\\Happy',
 'Emotion/Train\\Neutral',
 'Emotion/Train\\Sad',
 'Emotion/Train\\Surprise']

In [7]:
x = Flatten()(vgg.output)
x= Dense(units = 1500, kernel_initializer = 'he_normal',activation = 'relu')(x)
output = Dense(len(folders), activation = 'softmax')(x)

In [8]:
model = Model(inputs = vgg.input, outputs = output)

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [10]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

# if i use loss = 'sparse_categorical_crossentropy'  i get the error while fit() out model...


In [11]:
train_datagen = ImageDataGenerator(rescale = 1.0/224,
                                     zoom_range = 0.2,
                                    fill_mode = 'nearest')

In [12]:
test_datagen = ImageDataGenerator(rescale = 1.0/224)

In [13]:
train_set = train_datagen.flow_from_directory(train_path,
                                             target_size=(224,224),
                                             class_mode = 'categorical'
                                             )

valid_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (224,224),
                                            class_mode='categorical')

Found 19049 images belonging to 5 classes.
Found 3684 images belonging to 5 classes.


In [14]:
history = model.fit(train_set,
            validation_data=valid_set,
         epochs=5,
         steps_per_epoch= train_set.n // train_set.batch_size,
         validation_steps = valid_set.n // valid_set.batch_size)

Epoch 1/5
595/595 [==============================] - 4190s 7s/step - loss: 1.4640 - accuracy: 0.5185 - val_loss: 1.3589 - val_accuracy: 0.4927
Epoch 2/5
595/595 [==============================] - 3996s 7s/step - loss: 1.0205 - accuracy: 0.5926 - val_loss: 1.1594 - val_accuracy: 0.5861
Epoch 3/5
595/595 [==============================] - 4236s 7s/step - loss: 0.9686 - accuracy: 0.6170 - val_loss: 1.6468 - val_accuracy: 0.4818
Epoch 4/5
595/595 [==============================] - 4508s 8s/step - loss: 0.9446 - accuracy: 0.6265 - val_loss: 1.0762 - val_accuracy: 0.6266
Epoch 5/5
595/595 [==============================] - 4031s 7s/step - loss: 0.9069 - accuracy: 0.6437 - val_loss: 1.3563 - val_accuracy: 0.5416


# Saving our model

In [15]:
model.save('Facial Emotion detection.h5')

# load model

In [16]:
from tensorflow.keras.models import load_model

In [18]:
Model = load_model('Facial Emotion detection.h5')
Model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [19]:
import numpy
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [35]:
img = load_img('Emotion/validation/Happy/25.jpg', target_size =(224,224))
img = img_to_array(img)
img = img.reshape((1,) + input_shape)
img.shape

(1, 224, 224, 3)

In [36]:
def get_pred_value(img):
    pred = Model.predict(img)
    pred_item = np.argmax(pred, axis= -1)
    dict_ = train_set.class_indices
    labels = dict((m,n) for n,m in dict_.items())
    return labels[pred_item[0]]

prediction = get_pred_value(img)
print(prediction)

Happy


In [21]:
train_set.class_indices

{'Angry': 0, 'Happy': 1, 'Neutral': 2, 'Sad': 3, 'Surprise': 4}

In [27]:
pred_item = np.argmax(pred, axis= -1)
pred_item  #it returns an numpy array, so pred_item[0] - gives the integer value present in it..

array([4], dtype=int64)

In [32]:
dict_ = train_set.class_indices
labels = dict((m,n) for n,m in dict_.items())
labels[pred_item[0]]

'Surprise'